# Building an Image Classification Model Using AlexNet

In [1]:
import os
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

/Users/pvishnoi/PycharmProjects/data-etl-ml/venv/lib/python3.9/site-packages/torch/utils/_pytree.py:185: FutureWarning: optree is installed but the version is too old to support PyTorch Dynamo in C++ pytree. C++ pytree support is disabled. Please consider upgrading optree using `python3 -m pip install --upgrade 'optree>=0.13.0'`.
  warnings.warn(


In [2]:
# Define transformations for the training and validation sets
transform = transforms.Compose([
    transforms.Resize((227, 227)), #Alexnet expects an input size of 227 x 227 x 3
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.5,0.5,0.5), std=(0.5,0.5,0.5)) 
])

In [3]:
# Paths to the data
train_data_path = '/Users/pvishnoi/PycharmProjects/data-etl-ml/ml/GenAI-Pinnacle-Master/Computer Vision using PyTorch/Module 2/Human Action Recognition//train'
test_data_path = '/Users/pvishnoi/PycharmProjects/data-etl-ml/ml/GenAI-Pinnacle-Master/Computer Vision using PyTorch/Module 2/Human Action Recognition//test'

In [4]:
#Apply the transforms to the train and test data
train_data = datasets.ImageFolder(root=train_data_path, transform=transform)
test_data = datasets.ImageFolder(root=test_data_path, transform=transform)

##### Run all the cells above till here

### AlexNet Architecture

In [6]:
import torch.nn as nn
import torch.nn.functional as F

class AlexNet(nn.Module):
    def __init__(self, num_classes=5):
        super(AlexNet, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=11, stride=4, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(64, 192, kernel_size=5, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(192, 384, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(384, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
        )
        
        # Calculate the size of the feature map after the last pooling layer
        self.fc_input_dim = 256 * 6 * 6  # This is for input size 224x224
        
        self.classifier = nn.Sequential(
            nn.Dropout(),
            nn.Linear(self.fc_input_dim, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Linear(4096, num_classes),
        )

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)  # Flatten the tensor
        x = self.classifier(x)
        return x

Changes in the architecture are as follows:


1)__Increased Depth of Convolutional Layers:__ The AlexNet model uses more layers and deeper configurations. Starting with fewer feature channels, it progressively increases the depth across its layers. This contrasts sharply with LeNet, which uses a much simpler and shallower layer structure.


2)__Use of Batch Normalization:__ Each convolutional layer in AlexNet is followed by batch normalization, which helps in stabilizing the learning process by normalizing the input layer by adjusting and scaling activations. LeNet does not include batch normalization, which makes AlexNet better suited for training on larger datasets and more complex models.


3)__Advanced Activation Functions:__ AlexNet incorporates the ReLU (Rectified Linear Unit) activation function throughout its architecture, which helps to solve the vanishing gradient problem common in networks using sigmoid or tanh functions. LeNet traditionally uses sigmoid or tanh, which are less effective at preserving gradients during deep learning training.


4)__Increased and Varied Pooling Layers:__ AlexNet includes multiple max pooling layers that perform downsampling operation more frequently compared to LeNet. This reduces the spatial size of the representation, decreases the number of parameters and computation in the network, and hence, also controls overfitting.


5)__Integration of Dropout:__ In the classifier section of AlexNet, dropout layers are introduced. Dropout is a form of regularization that helps in preventing overfitting by randomly setting a fraction of input units to 0 at each update during training time, which helps in making the model robust and less likely to rely on any one feature.


6)__Complex Classifier Structure:__ AlexNet's classifier is more complex, using three fully connected layers with dropout and ReLU activation functions between them. This is a significant escalation from LeNet’s typically one or two-layer classifiers. This complexity allows AlexNet to make more fine-grained classifications based on the richer feature hierarchies developed in the convolutional layers.


7)__Scalability to Higher Resolution Inputs:__ AlexNet is structured to handle higher resolution input images better than LeNet. The initial convolutional layer uses a larger receptive field (11x11 with a stride of 4), which is suitable for larger input images, whereas LeNet is optimized for smaller images typical of earlier datasets like MNIST.


In [7]:
# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [8]:
device

device(type='cpu')

In [9]:
# Initialize the model
model = AlexNet().to(device)

In [10]:
import torch.optim as optim

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters())

In [11]:
# Create DataLoader for training and validation sets
train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
test_loader = DataLoader(test_data, batch_size=32, shuffle=False)

In [ ]:
import torch
from tqdm import tqdm
import matplotlib.pyplot as plt

# Initialize the lists to store train and test loss for each epoch
train_losses = []
test_losses = []

# Train the model
num_epochs = 20
best_loss = torch.inf
patience = 5
epochs_since_best = 0

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for images, labels in tqdm(train_loader):
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        if isinstance(outputs, tuple):
            outputs = outputs[0]  # For models that return auxiliary outputs
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()

    train_loss = running_loss / len(train_loader)
    train_losses.append(train_loss)  # Store the train loss for this epoch
    train_accuracy = 100. * correct / total

    print(f'Epoch [{epoch + 1}/{num_epochs}], '
          f'Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.2f}%')

    # Evaluate on the test set
    model.eval()
    test_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            if isinstance(outputs, tuple):
                outputs = outputs[0]  # For models that return auxiliary outputs
            loss = criterion(outputs, labels)
            test_loss += loss.item()
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()

    test_loss /= len(test_loader)
    test_losses.append(test_loss)  # Store the test loss for this epoch
    test_accuracy = 100. * correct / total

    print(f'Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.2f}%')

    # Check for best accuracy and stop if not improved after five more epochs
    if test_loss < best_loss:
        best_loss = test_loss
        epochs_since_best = 0
        torch.save(model.state_dict(), 'best_model.pth')  # Save the model
        print(f'Updated best model with accuracy: {test_accuracy:.2f}%')
    else:
        epochs_since_best += 1
        if epochs_since_best > patience:
            print("Stopping early: no improvement after five consecutive epochs.")
            break

100%|██████████| 112/112 [00:58<00:00,  1.91it/s]


Epoch [1/20], Train Loss: 1.6152, Train Accuracy: 21.82%
Test Loss: 1.6000, Test Accuracy: 23.17%
Updated best model with accuracy: 23.17%


100%|██████████| 112/112 [01:02<00:00,  1.80it/s]


Epoch [2/20], Train Loss: 1.5954, Train Accuracy: 24.20%
Test Loss: 1.5791, Test Accuracy: 22.22%
Updated best model with accuracy: 22.22%


 21%|██▏       | 24/112 [00:13<00:47,  1.84it/s]

### Plot: Epochs vs Train Loss and Test

In [ ]:
# Plotting the training and validation losses
plt.plot(train_losses, label='Training Loss')
plt.plot(test_losses, label='Test Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()